In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
try:
    df = pd.read_csv('/datasets/real_estate_data.csv', sep='\t')
except:
    df = pd.read_csv('https://code.s3.yandex.net/datasets/real_estate_data.csv', sep='\t')
df.head()

,total_images,last_price,total_area,first_day_exposition,rooms,ceiling_height,floors_total,living_area,floor,is_apartment,...,kitchen_area,balcony,locality_name,airports_nearest,cityCenters_nearest,parks_around3000,parks_nearest,ponds_around3000,ponds_nearest,days_exposition
0,20,13000000.0,108.0,2019-03-07T00:00:00,3,2.70,16.0,51.0,8,NaN,...,25.0,NaN,Санкт-Петербург,18863.0,16028.0,1.0,482.0,2.0,755.0,NaN
1,7,3350000.0,40.4,2018-12-04T00:00:00,1,NaN,11.0,18.6,1,NaN,...,11.0,2.0,посёлок Шушары,12817.0,18603.0,0.0,NaN,0.0,NaN,81.0
2,10,5196000.0,56.0,2015-08-20T00:00:00,2,NaN,5.0,34.3,4,NaN,...,8.3,0.0,Санкт-Петербург,21741.0,13933.0,1.0,90.0,2.0,574.0,558.0
3,0,64900000.0,159.0,2015-07-24T00:00:00,3,NaN,14.0,NaN,9,NaN,...,NaN,0.0,Санкт-Петербург,28098.0,6800.0,2.0,84.0,3.0,234.0,424.0
4,2,10000000.0,100.0,2018-06-19T00:00:00,2,3.03,14.0,32.0,13,NaN,...,41.0,NaN,Санкт-Петербург,31856.0,8098.0,2.0,112.0,1.0,48.0,121.0


In [50]:
#df.loc[19904, 'kitchen_area'] = 2
df.loc[:,['total_area', 'living_area']].sort_values(by='total_area', ascending=True).head(10)

,total_area,living_area
19904,12.0,10.0
17961,13.0,NaN
19546,13.0,NaN
19807,13.0,10.0
19558,13.2,NaN
19642,14.0,11.0
16949,15.0,NaN
12040,15.0,NaN
8886,15.5,10.0
9412,16.0,13.0


In [ ]:
tmp = df.groupby('total_area')['living_area'].median().dropna()

df.loc[df['living_area'].isna(), 'living_area'] = \
pd.merge_asof(df['total_area'].reset_index(), tmp, on='total_area').set_index('index')['living_area']

# Рабочий код

In [52]:
data = pd.DataFrame({'total_area': [5, 8, 8, 8, 20, 56, 59], \
'living_area': [40, 51, 53, np.nan, np.nan, 34, np.nan]})

tmp = data.groupby('total_area')['living_area'].median().dropna()

data.loc[data['living_area'].isna(), 'living_area'] = \
pd.merge_asof(data['total_area'].reset_index(), tmp, on='total_area').set_index('index')['living_area']

In [51]:
data.loc[:,['total_area', 'living_area']].sort_values(by='total_area', ascending=True).head(10)

,total_area,living_area
0,5,40.0
1,8,51.0
2,8,53.0
3,8,52.0
4,20,52.0
5,56,34.0
6,59,34.0
